In [25]:
import pandas as pd
import numpy as np

#Used to allow the browser to load. 
import time
from tqdm import tqdm

#used as a import for dateing the CSV
import datetime

#NLP libaries
import spacy
from spacy.lang.en import STOP_WORDS

import gensim
import pyLDAvis.gensim
from gensim.models import LdaModel
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import coherencemodel
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

## Using Topic Modeling using Gensim
Objective: Setup for topic modeling and use LDA to determine feature importance

In [26]:
job_list = pd.read_csv('./data/2018-06-25_Data Scientist_Austin, TX job_list.csv')

In [39]:
nlp = spacy.load('en', disable=['parser', 'ner'])

In [40]:
pos_description = job_list['position_description'].values

In [41]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(simple_preprocess(str(pos_description),deacc=True))

In [42]:
data_words = list(sent_to_words(pos_description))

In [43]:
#Building bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=4, threshold=100)

trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

In [44]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [45]:
def remove_stopwords(text):
    return [[word for word in simple_preprocess(str(doc)) if word not in STOP_WORDS] for doc in text]

def make_bigrams(text):
    return [bigram_mod[doc] for doc in text]

def make_trigrams(text):
    return [trigram_mod[bigram_mod[doc]] for doc in text]

def lemmatization(text, allowed_postags = ['NOUN','ADJ','VERB','ADV']):
    text_rem = []
    for sent in text:
        doc = nlp(" ".join(sent))
        text_rem.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        return text_rem 

In [46]:
remove_more_words = ['skill','datum','day','week','year','ability','analyst','recovery','austin','document','ago']

In [47]:
words_no_stops = remove_stopwords(data_words)

word_bigrams = make_bigrams(words_no_stops)

data_lemma =  lemmatization(word_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [48]:
data_lemma[0] = [word for word in data_lemma[0] if word not in remove_more_words]

In [49]:
id2word = corpora.Dictionary(data_lemma)

texts = data_lemma

corpus = [id2word.doc2bow(text) for text in texts]

In [50]:
data_lemma

[['client',
  'company',
  'use',
  'artificial',
  'intelligence',
  'machine',
  'learning',
  'technique',
  'build',
  'software',
  'system',
  'think',
  'adapt',
  'interact',
  'people',
  'quality',
  'assurancelead',
  'responsible',
  'define',
  'shaping',
  'testing',
  'methofology_alongside',
  'engineering',
  'leadership',
  'work',
  'engineering',
  'leadership',
  'shape',
  'define',
  'test',
  'methodology',
  'work',
  'closely',
  'solution',
  'architect',
  'project',
  'manager',
  'customer',
  'define',
  'work',
  'item',
  'acceptance',
  'criterion',
  'write',
  'execute',
  'test',
  'plan',
  'integrate',
  'test',
  'suite',
  'pipeline',
  'work',
  'closely',
  'machine',
  'learn',
  'engineer',
  'scientist',
  'craft',
  'testing',
  'methodology',
  'ai',
  'produce',
  'thoroughly',
  'detailed',
  'defect',
  'report',
  'participate',
  'code',
  'review',
  'necessary',
  'work',
  'understand',
  'code',
  'testing',
  'prepare',
  'sprin

In [51]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word,num_topics=10,
                               random_state=42,
                               update_every=1,
                               chunksize=100,
                               passes=10,
                               alpha='auto',
                               per_word_topics=False)

In [52]:
doc_lda = lda_model[corpus]

In [55]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = coherencemodel.CoherenceModel(model=lda_model, texts=data_lemma , dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.341735867717185

Coherence Score:  0.3090886691580874


In [56]:
pyLDAvis.gensim.prepare(lda_model,corpus, id2word)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
5      96.003181        1       1 -0.159316  0.012087
0       3.996756        1       2 -0.129524 -0.015843
4       0.000011        1       3  0.051170  0.000350
2       0.000008        1       4  0.045338  0.000271
8       0.000007        1       5  0.044216 -0.000452
3       0.000007        1       6  0.042778 -0.000314
9       0.000006        1       7  0.041030 -0.000614
6       0.000005        1       8  0.039738 -0.000524
7       0.000004        1       9  0.031581 -0.001082
1       0.000002        1      10 -0.007010  0.006121, topic_info=     Category          Freq         Term       Total  loglift  logprob
term                                                                  
1356  Default  5.570000e+02   experience  557.000000  30.0000  30.0000
1089  Default  5.450000e+02          ago  545.000000  29.0000  29.0000
559   Default  5.060000e+02          job  506.000000  28.0000  28.0000
2294  Default  4.630000e+02         work  463.000000  27.0000  27.0000
1630  Default  3.530000e+02    scientist  353.000000  26.0000  26.0000
1054  Default  3.080000e+02         team  308.000000  25.0000  25.0000
392   Default  3.040000e+02        apply  304.000000  24.0000  24.0000
519   Default  2.670000e+02     business  267.000000  23.0000  23.0000
114   Default  2.370000e+02     engineer  237.000000  22.0000  22.0000
2362  Default  2.340000e+02      company  234.000000  21.0000  21.0000
169   Default  2.520000e+02     analytic  252.000000  20.0000  20.0000
570   Default  2.220000e+02     software  222.000000  19.0000  19.0000
1306  Default  1.980000e+02     research  198.000000  18.0000  18.0000
2173  Default  1.790000e+02         data  179.000000  17.0000  17.0000
762   Default  1.970000e+02      science  197.000000  16.0000  16.0000
211   Default  1.670000e+02      require  167.000000  15.0000  15.0000
563   Default  1.800000e+02   technology  180.000000  14.0000  14.0000
879   Default  1.690000e+02      product  169.000000  13.0000  13.0000
2134  Default  1.530000e+02  opportunity  153.000000  12.0000  12.0000
3437  Default  1.630000e+02  development  163.000000  11.0000  11.0000
3560  Default  1.420000e+02      project  142.000000  10.0000  10.0000
97    Default  1.490000e+02     customer  149.000000   9.0000   9.0000
984   Default  1.410000e+02       senior  141.000000   8.0000   8.0000
648   Default  1.380000e+02      develop  138.000000   7.0000   7.0000
906   Default  1.350000e+02     solution  135.000000   6.0000   6.0000
2367  Default  1.390000e+02  application  139.000000   5.0000   5.0000
1741  Default  1.280000e+02       easily  128.000000   4.0000   4.0000
3569  Default  1.380000e+02     analysis  138.000000   3.0000   3.0000
1246  Default  1.360000e+02        learn  136.000000   2.0000   2.0000
1192  Default  1.360000e+02       design  136.000000   1.0000   1.0000
...       ...           ...          ...         ...      ...      ...
1630  Topic10  1.729707e-06    scientist  353.635193  -1.3374  -5.9796
2362  Topic10  1.260823e-06      company  234.879745  -1.2444  -6.2957
392   Topic10  1.363644e-06        apply  304.673218  -1.4262  -6.2173
2173  Topic10  1.125268e-06         data  179.929886  -1.0917  -6.4095
114   Topic10  1.158141e-06     engineer  237.985123  -1.3425  -6.3807
211   Topic10  1.025645e-06      require  167.333115  -1.1118  -6.5022
570   Topic10  1.100078e-06     software  222.782410  -1.3279  -6.4321
648   Topic10  8.769795e-07      develop  138.347366  -1.0781  -6.6588
519   Topic10  1.045369e-06     business  267.127838  -1.5605  -6.4831
1741  Topic10  8.329082e-07       easily  128.257797  -1.0540  -6.7103
984   Topic10  8.268270e-07       senior  141.382339  -1.1587  -6.7177
3766  Topic10  7.529200e-07      process  104.333641  -0.9485  -6.8113
762   Topic10  9.063762e-07      science  197.055466  -1.3989  -6.6258
169   Topic10  9.677458e-07     analytic  2